In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_3243673/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_3_8_21,0.961253,0.912872,0.912830,0.921499,4.672874,15.925322,11.073429,13.642085,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
1,model_3_8_22,0.959928,0.911831,0.904533,0.918065,4.832736,16.115511,12.127414,14.238761,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
2,model_3_8_23,0.959563,0.912469,0.899721,0.916765,4.876718,15.998988,12.738694,14.464738,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
3,model_3_8_17,0.958110,0.921746,0.914453,0.926998,5.052003,14.303308,10.867223,12.686318,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
4,model_3_8_20,0.958099,0.916427,0.899459,0.918879,5.053307,15.275438,12.771943,14.097336,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
5,model_3_8_18,0.957612,0.918855,0.903120,0.921490,5.112047,14.831670,12.306893,13.643538,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
6,model_3_8_19,0.957605,0.918619,0.901459,0.920787,5.112810,14.874905,12.517935,13.765743,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
7,model_3_8_24,0.957261,0.910771,0.886202,0.911169,5.154382,16.309414,14.456057,15.437240,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
8,model_3_8_16,0.956402,0.920601,0.914376,0.926334,5.257893,14.512585,10.877048,12.801738,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
9,model_3_9_0,0.955616,0.851334,0.923597,0.920371,5.352747,4.528910,11.858478,7.978176,"Hidden Size=[70, 35], regularizer=0.1, learnin..."
